In [ ]:
import sys
import os
import psycopg2
import pandas as pd
import requests
from IPython.display import display
sys.path.insert(0, os.path.realpath('..'))
import django
django.setup()

### Overview

Get a list of stations, pull the response zones from geo.firecares.org, convert to shapefiles and merge into a single shapefile: `/tmp/outf.shp`

In [ ]:
stations = FireStation.objects.filter(state='TN', department__name__contains='Memphis')
print stations.count()

In [ ]:
host = """https://geo.firecares.org/?"""

qs = {
    'f': 'kmz',
    'Facilities': {
        'features': [
            {
                'geometry': {
                    'x': -90.0455557969999,
                    'y': 35.15284569100004,
                    'spatialReference': {
                        'wkid': 4326
                    }
                },
            }
        ],
        'geometryType': 'esriGeometryPoint'
    },
    'env:outSR': 4326,
    'text_input': '4,4,4',
    'Break_Values': '4 6 8',
    'returnZ': False,
    'returnM': False
}

from firecares.firestation.models import FireStation
import urllib
import copy
print stations.count()
files = []

for fs in stations:
    q = copy.deepcopy(qs)
    q['Facilities']['features'][0]['geometry']['x'] = fs.geom.x
    q['Facilities']['features'][0]['geometry']['y'] = fs.geom.y
    print '{}{}'.format(host, q)
    url = host + urllib.urlencode(q)
    resp = requests.get(url)
    fname = '/tmp/{}.kmz'.format(fs.id)
    tmp_shp = '/tmp/{}.shp'.format(fs.id)
    files.append(tmp_shp)
    with open(fname, 'w') as f:
        f.write(resp.content)  
    !ogr2ogr -f "ESRI Shapefile" {tmp_shp} {fname} -dim 2
    !ogrinfo /tmp/{fs.id}.shp -sql 'alter table "{fs.id}" add column fc_id integer'
    !ogrinfo /tmp/{fs.id}.shp -dialect SQLite -sql 'update "{fs.id}" set fc_id = {fs.id}'
    !ogrinfo /tmp/{fs.id}.shp -sql 'alter table "{fs.id}" add column dept_fdid character(5)'
    !ogrinfo /tmp/{fs.id}.shp -dialect SQLite -sql "update \"{fs.id}\" set dept_fdid = '{fs.department.fdid}'"
    !ogrinfo /tmp/{fs.id}.shp -sql 'alter table "{fs.id}" add column st_name character(254)'
    !ogrinfo /tmp/{fs.id}.shp -dialect SQLite -sql "update \"{fs.id}\" set st_name = '{fs.name}'"
    

In [ ]:
!ogr2ogr /tmp/outf.shp {files[0]}

for f in files[1:]:
    !ogr2ogr -update -append /tmp/outf.shp {f}

In [ ]:
!ogrinfo /tmp/outf.shp -al